In [1]:
from dask_image.imread import imread
from skimage import io
import napari
import pycudadecon
import numpy as np


In [2]:
psf = '/Users/talley/Dropbox (HMS)/CBMF/lattice_sample_data/lls_PSFs/488_psf.tif'
psf = io.imread(psf)

stacks = imread('/Volumes/extra850/mitosis/*ch1*.tif')
stacks

/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")


,Array,Chunk
Bytes,5.11 GB,8.52 MB
Shape,"(600, 65, 256, 256)","(1, 65, 256, 256)"
Count,1800 Tasks,600 Chunks
Type,uint16,numpy.ndarray


In [4]:
def deskew(a):
    if not a.ndim > 3:
        return a
    r = pycudadecon.deskew_gpu(a[0], angle=31, pad_val=0)
    return np.expand_dims(r,0)

deskewed = stacks.map_blocks(deskew, dtype='uint16')


def decon(a):
    if not a.ndim > 3:
        return a
    r = pycudadecon.decon(a[0], psf, background=90)[:,:,:-1]
    return np.expand_dims(np.ascontiguousarray(r),0)
deconvolved = deskewed.map_blocks(decon, dtype='uint16')


In [18]:
from functools import partial

def last3dims(f):
    def func(array):
        return np.expand_dims(f(array[0]), 0)
    return func

deskew = last3dims(partial(pycudadecon.deskew_gpu, angle=31.5))
deconv = last3dims(partial(pycudadecon.decon, psf=psf, deskew=31.5, background=90))

def crop(array):
    return array[:,2:,10:-20,:500]


deskewed = stacks.map_blocks(deskew, dtype='uint16')
deconvolved = stacks.map_blocks(deconv, dtype='float32')
cropped = deconvolved.map_blocks(crop, dtype='float32')

In [ ]:
with napari.gui_qt():
    v = napari.view_image(cropped,
                      contrast_limits=[90,1500],
                      is_pyramid=False,
                      scale=(3,1,1),
                      translate=(0,-150),
                      ndisplay=3
                     )
    v.window._qt_window.raise_()

/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")
/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")
/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")
/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")
/Users/talley/miniconda3/envs/all/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2133: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")
/Users/talley/minico

In [9]:
io.imsave('test.tif', i)

In [10]:
q = io.imread('test.tif')

In [11]:
with napari.gui_qt():
    v = napari.view_image(q)